In [1]:
import pygame
import time

pygame.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
BACKGROUND_IMAGE = 'background.png'

window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Игра v1.0")

background_img = pygame.image.load(BACKGROUND_IMAGE)
background = pygame.transform.scale(background_img, (WINDOW_WIDTH, WINDOW_HEIGHT))

speed = 0
background_offset = 0

running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                speed = 5

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                speed = 0

    background_offset = (background_offset + speed) % WINDOW_WIDTH

    window.blit(background, (background_offset, 0))

    if background_offset != 0:
        window.blit(background, (background_offset - WINDOW_WIDTH, 0))
    
    pygame.display.update()
    clock.tick(60)

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
BACKGROUND_IMAGE = 'background.png'

window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Игра v1.0")

speed = {'left': 0, 'right': 0}
background_offset = 0

running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                speed['left'] = 5
            elif event.key == pygame.K_RIGHT:
                speed['right'] = -5

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                speed['left'] = 0
            elif event.key == pygame.K_RIGHT:
                speed['right'] = 0

    total_speed = speed['left'] + speed['right']
    background_offset = (background_offset + total_speed) % WINDOW_WIDTH

    window.blit(background, (background_offset, 0))

    if background_offset > 0:
        window.blit(background, (background_offset - WINDOW_WIDTH, 0))
    elif background_offset < 0:
        window.blit(background, (background_offset + WINDOW_WIDTH, 0))

    pygame.display.update()
    clock.tick(60)

pygame.quit()

In [3]:
class Player(pygame.sprite.Sprite):
    def __init__(self, filename, hero_x=100, hero_y=250, x_speed=0, y_speed=0):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load(filename)
        self.rect = self.image.get_rect()

        self.hero_x = hero_x
        self.hero_y = hero_y
        self.rect.x = hero_x
        self.rect.y = hero_y

        self.x_speed = x_speed
        self.y_speed = y_speed
    
    def update(self):
        """Перемещает персонажа с учетом текущей скорости"""
        self.hero_x += self.x_speed
        self.hero_y += self.y_speed
        self.rect.x = self.hero_x
        self.rect.y = self.hero_y

if __name__ == "__main__":
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    
    filename = 'player.png'
    hero = Player(filename)
    print(hero.__dict__)
    
    running = True
    clock = pygame.time.Clock()
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        hero.update()

        screen.fill((0, 0, 0))
        screen.blit(hero.image, hero.rect)
        pygame.display.flip()
        
        clock.tick(60)
    
    pygame.quit()

{'_Sprite__g': set(), 'image': <Surface(212x190x8 SW)>, 'rect': <rect(100, 250, 212, 190)>, 'hero_x': 100, 'hero_y': 250, 'x_speed': 0, 'y_speed': 0}


In [4]:
import sys

pygame.init()

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
BG_COLOR = (0, 0, 0)
PLAYER_SPEED = 5
BG_SCROLL_SPEED = 3

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Аркада v2.0")

class Player(pygame.sprite.Sprite):
    def __init__(self, x=100, y=300):
        super().__init__()
        self.image = pygame.image.load("player.png").convert_alpha()

        
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.speed_x = 0
        self.speed_y = 0
    
    def update(self, bg_offset):
        self.rect.x += self.speed_x
        self.rect.y += self.speed_y

        if self.rect.top < 0:
            self.rect.top = 0
        if self.rect.bottom > SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

        at_left_edge = self.rect.left < 50 and self.speed_x < 0
        at_right_edge = self.rect.right > SCREEN_WIDTH - 50 and self.speed_x > 0
        
        return at_left_edge or at_right_edge


bg_image = pygame.image.load("background.png").convert()
bg_image = pygame.transform.scale(bg_image, (SCREEN_WIDTH * 2, SCREEN_HEIGHT))

player = Player()
all_sprites = pygame.sprite.Group(player)

bg_offset = 0
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                player.speed_x = -PLAYER_SPEED
            if event.key == pygame.K_RIGHT:
                player.speed_x = PLAYER_SPEED
            if event.key == pygame.K_UP:
                player.speed_y = -PLAYER_SPEED
            if event.key == pygame.K_DOWN:
                player.speed_y = PLAYER_SPEED

        if event.type == pygame.KEYUP:
            if event.key in (pygame.K_LEFT, pygame.K_RIGHT):
                player.speed_x = 0
            if event.key in (pygame.K_UP, pygame.K_DOWN):
                player.speed_y = 0

    should_scroll = player.update(bg_offset)

    if should_scroll:
        bg_offset -= player.speed_x
        player.rect.x -= player.speed_x
    
    screen.blit(bg_image, (bg_offset % (SCREEN_WIDTH * 2) - SCREEN_WIDTH * 2, 0))
    screen.blit(bg_image, (bg_offset % (SCREEN_WIDTH * 2), 0))

    all_sprites.draw(screen)
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
import random

pygame.init()

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
PLAYER_SPEED = 5
METEOR_SPEED = 2
SHOT_SPEED = 10

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Аркада с графикой")


background = pygame.image.load("background.png").convert()
background = pygame.transform.scale(background, (SCREEN_WIDTH, SCREEN_HEIGHT))


class Player(pygame.sprite.Sprite):
    def __init__(self, x=100, y=300):
        super().__init__()
        self.image = pygame.image.load("player.png").convert_alpha()
        self.image = pygame.transform.scale(self.image, (50, 50))

        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.speed_x = 0
        self.speed_y = 0
    
    def update(self):
        self.rect.x += self.speed_x
        self.rect.y += self.speed_y

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top < 0:
            self.rect.top = 0
        if self.rect.bottom > SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

class Meteor(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("meteor.png").convert_alpha()
        self.image = pygame.transform.scale(self.image, (80, 80))
        
        self.rect = self.image.get_rect()
        self.rect.x = random.randrange(SCREEN_WIDTH - self.rect.width)
        self.rect.y = random.randrange(SCREEN_HEIGHT - self.rect.height)
        self.speed_x = random.choice([-1, 1]) * METEOR_SPEED
        self.speed_y = random.choice([-1, 1]) * METEOR_SPEED
    
    def update(self):
        self.rect.x += self.speed_x
        self.rect.y += self.speed_y

        if self.rect.left < 0 or self.rect.right > SCREEN_WIDTH:
            self.speed_x *= -1
        if self.rect.top < 0 or self.rect.bottom > SCREEN_HEIGHT:
            self.speed_y *= -1

class Shot(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.image.load("shot.png").convert_alpha()
        self.image = pygame.transform.scale(self.image, (30, 15))
        self.rect = self.image.get_rect()
        self.rect.left = x
        self.rect.centery = y
        self.speed = SHOT_SPEED
    
    def update(self):
        self.rect.x += self.speed
        if self.rect.left > SCREEN_WIDTH:
            self.kill()

player = Player()
enemies = pygame.sprite.Group()
for i in range(5):
    enemies.add(Meteor())

shots = pygame.sprite.Group()

all_sprites = pygame.sprite.Group()
all_sprites.add(player)
all_sprites.add(enemies)

clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                player.speed_x = -PLAYER_SPEED
            if event.key == pygame.K_RIGHT:
                player.speed_x = PLAYER_SPEED
            if event.key == pygame.K_UP:
                player.speed_y = -PLAYER_SPEED
            if event.key == pygame.K_DOWN:
                player.speed_y = PLAYER_SPEED
            if event.key == pygame.K_SPACE:
                shot = Shot(player.rect.right, player.rect.centery)
                shots.add(shot)
                all_sprites.add(shot)
        
        if event.type == pygame.KEYUP:
            if event.key in (pygame.K_LEFT, pygame.K_RIGHT):
                player.speed_x = 0
            if event.key in (pygame.K_UP, pygame.K_DOWN):
                player.speed_y = 0

    player.update()
    enemies.update()
    shots.update()

    hits = pygame.sprite.groupcollide(shots, enemies, True, True)

    screen.blit(background, (0, 0))
    all_sprites.draw(screen)
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 